#### Performing Objective 3 -> To predict employees at the risk of under performance which leads to employee attrition
#### Use past and present employee information to predict which workers are likely to start underperforming soon. This lets managers take action early—such as offering training or support—before the person’s performance drops further or they decide to leave

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# Preprocessing and metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                           f1_score, confusion_matrix, classification_report)
from sklearn.model_selection import train_test_split, cross_val_score

# ML Models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

# For hyperparameter tuning
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Utilities
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [20]:
ibm_dataset = pd.read_csv('./dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')

hr_job_dataset = pd.read_csv('./dataset/HR_comma_sep.csv')

emp_attrition_dataset = pd.read_csv('./dataset/employee_attrition_data.csv')

In [3]:
## loading the ibm_dataset

ibm_dataset.head(10)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
5,32,No,Travel_Frequently,1005,Research & Development,2,2,Life Sciences,1,8,...,3,80,0,8,2,2,7,7,3,6
6,59,No,Travel_Rarely,1324,Research & Development,3,3,Medical,1,10,...,1,80,3,12,3,2,1,0,0,0
7,30,No,Travel_Rarely,1358,Research & Development,24,1,Life Sciences,1,11,...,2,80,1,1,2,3,1,0,0,0
8,38,No,Travel_Frequently,216,Research & Development,23,3,Life Sciences,1,12,...,2,80,0,10,2,3,9,7,1,8
9,36,No,Travel_Rarely,1299,Research & Development,27,3,Medical,1,13,...,2,80,2,17,3,2,7,7,7,7


In [4]:
## loading the hr_job_dataset

hr_job_dataset.head(10)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low
5,0.41,0.50,2,153,3,0,1,0,sales,low
6,0.10,0.77,6,247,4,0,1,0,sales,low
7,0.92,0.85,5,259,5,0,1,0,sales,low
8,0.89,1.00,5,224,5,0,1,0,sales,low
9,0.42,0.53,2,142,3,0,1,0,sales,low


In [21]:
## dropping the cols from the dataset which is irrelevant for the current use case

hr_job_dataset = hr_job_dataset.drop(['number_project', 'Work_accident'], axis=1)

In [6]:
hr_job_dataset.head(10)

,satisfaction_level,last_evaluation,average_montly_hours,time_spend_company,left,promotion_last_5years,Department,salary
0,0.38,0.53,157,3,1,0,sales,low
1,0.80,0.86,262,6,1,0,sales,medium
2,0.11,0.88,272,4,1,0,sales,medium
3,0.72,0.87,223,5,1,0,sales,low
4,0.37,0.52,159,3,1,0,sales,low
5,0.41,0.50,153,3,1,0,sales,low
6,0.10,0.77,247,4,1,0,sales,low
7,0.92,0.85,259,5,1,0,sales,low
8,0.89,1.00,224,5,1,0,sales,low
9,0.42,0.53,142,3,1,0,sales,low


In [22]:
## filtering the dataset based on only 3 departments -> HR, Sales and R&D to keep the data uniform

selected_depts = ['sales', 'hr', 'RandD']
hr_job_filtered_df = hr_job_dataset[hr_job_dataset['Department'].isin(selected_depts)]

In [8]:
### printing the filtered df

hr_job_filtered_df.head(15)

,satisfaction_level,last_evaluation,average_montly_hours,time_spend_company,left,promotion_last_5years,Department,salary
0,0.38,0.53,157,3,1,0,sales,low
1,0.80,0.86,262,6,1,0,sales,medium
2,0.11,0.88,272,4,1,0,sales,medium
3,0.72,0.87,223,5,1,0,sales,low
4,0.37,0.52,159,3,1,0,sales,low
5,0.41,0.50,153,3,1,0,sales,low
6,0.10,0.77,247,4,1,0,sales,low
7,0.92,0.85,259,5,1,0,sales,low
8,0.89,1.00,224,5,1,0,sales,low
9,0.42,0.53,142,3,1,0,sales,low


In [9]:
hr_job_filtered_df['Department'].value_counts()

Department
sales    4140
RandD     787
hr        739
Name: count, dtype: int64

In [10]:
hr_job_filtered_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
satisfaction_level,5666.0,0.613154,0.249288,0.09,0.44,0.64,0.82,1.0
last_evaluation,5666.0,0.709938,0.170975,0.36,0.55,0.71,0.86,1.0
average_montly_hours,5666.0,200.605542,49.620550,96.00,156.00,200.00,245.00,310.0
time_spend_company,5666.0,3.487646,1.435131,2.00,3.00,3.00,4.00,10.0
left,5666.0,0.238263,0.426059,0.00,0.00,0.00,0.00,1.0
promotion_last_5years,5666.0,0.025062,0.156327,0.00,0.00,0.00,0.00,1.0


In [23]:
## Renaming the department values


hr_job_filtered_df['Department'] = hr_job_filtered_df['Department'].replace({
    'sales': 'Sales',
    'hr': 'Human Resources',
    'RandD': 'Research & Development'
})

hr_job_filtered_df['Department'].value_counts()

Department
Sales                     4140
Research & Development     787
Human Resources            739
Name: count, dtype: int64

In [24]:
## Renaming the satisfaction_level to JobSatisfactionLevel

hr_job_filtered_df = hr_job_filtered_df.rename(columns={'satisfaction_level': 'JobSatisfaction'})
hr_job_filtered_df.head(10)

,JobSatisfaction,last_evaluation,average_montly_hours,time_spend_company,left,promotion_last_5years,Department,salary
0,0.38,0.53,157,3,1,0,Sales,low
1,0.80,0.86,262,6,1,0,Sales,medium
2,0.11,0.88,272,4,1,0,Sales,medium
3,0.72,0.87,223,5,1,0,Sales,low
4,0.37,0.52,159,3,1,0,Sales,low
5,0.41,0.50,153,3,1,0,Sales,low
6,0.10,0.77,247,4,1,0,Sales,low
7,0.92,0.85,259,5,1,0,Sales,low
8,0.89,1.00,224,5,1,0,Sales,low
9,0.42,0.53,142,3,1,0,Sales,low


In [25]:
### applying a logic of filtering condition for the job satisfaction level because it has to match the ibm dataset 

### To achieve -> 1 => "Low", 2 => "Medium", 3 => "High", 4 => "Very High"
### Criteria to apply -> <0.30 => Low, <0.50 => Medium , <0.80 => High, > 0.80 => Very High

def filtering_condition_job_satisfaction(val):
  if val < 0.35:
    return 1
  elif val >= 0.30 and val < 0.50:
    return 2
  elif val >= 0.50 and val < 0.80:
    return 3
  else:
    return 4

hr_job_filtered_df["JobSatisfaction"] = hr_job_filtered_df["JobSatisfaction"].apply(lambda x: filtering_condition_job_satisfaction(x))

In [14]:
hr_job_filtered_df.head(10)

,JobSatisfaction,last_evaluation,average_montly_hours,time_spend_company,left,promotion_last_5years,Department,salary
0,Medium,0.53,157,3,1,0,Sales,low
1,Very High,0.86,262,6,1,0,Sales,medium
2,Low,0.88,272,4,1,0,Sales,medium
3,High,0.87,223,5,1,0,Sales,low
4,Medium,0.52,159,3,1,0,Sales,low
5,Medium,0.50,153,3,1,0,Sales,low
6,Low,0.77,247,4,1,0,Sales,low
7,Very High,0.85,259,5,1,0,Sales,low
8,Very High,1.00,224,5,1,0,Sales,low
9,Medium,0.53,142,3,1,0,Sales,low


In [26]:
## Renaming the coln name time_spend_company to YearsAtCompany
hr_job_filtered_df = hr_job_filtered_df.rename(columns={'time_spend_company': 'YearsAtCompany'})


In [27]:
## changing the col name 'left' to Attrition name
hr_job_filtered_df = hr_job_filtered_df.rename(columns={'left': 'Attrition'})

In [28]:
hr_job_filtered_df['Attrition'] = hr_job_filtered_df['Attrition'].replace({
    1: 'Yes',
    0: 'No'
})

In [18]:
hr_job_filtered_df.head(10)

,JobSatisfaction,last_evaluation,average_montly_hours,YearsAtCompany,Attrition,promotion_last_5years,Department,salary
0,Medium,0.53,157,3,Yes,0,Sales,low
1,Very High,0.86,262,6,Yes,0,Sales,medium
2,Low,0.88,272,4,Yes,0,Sales,medium
3,High,0.87,223,5,Yes,0,Sales,low
4,Medium,0.52,159,3,Yes,0,Sales,low
5,Medium,0.50,153,3,Yes,0,Sales,low
6,Low,0.77,247,4,Yes,0,Sales,low
7,Very High,0.85,259,5,Yes,0,Sales,low
8,Very High,1.00,224,5,Yes,0,Sales,low
9,Medium,0.53,142,3,Yes,0,Sales,low


In [29]:
## renaming the last_evaluation to PerformanceRating

hr_job_filtered_df = hr_job_filtered_df.rename(columns={'last_evaluation': 'PerformanceRating'})

### To achieve -> 1 => "Low", 2 => "Good", 3 => "Excellent", 4 => "Outstanding"
### Criteria to apply -> <0.30 => Low, <0.50 => Good , <0.80 => Excellent, > 0.80 => Outstanding

def filtering_condition_last_evaluation(val):
  if val < 0.35:
    return 1
  elif val >= 0.30 and val < 0.50:
    return 2
  elif val >= 0.50 and val < 0.80:
    return 3
  else:
    return 4

hr_job_filtered_df["PerformanceRating"] = hr_job_filtered_df["PerformanceRating"].apply(lambda x: filtering_condition_last_evaluation(x))
hr_job_filtered_df.head(10)

,JobSatisfaction,PerformanceRating,average_montly_hours,YearsAtCompany,Attrition,promotion_last_5years,Department,salary
0,2,3,157,3,Yes,0,Sales,low
1,4,4,262,6,Yes,0,Sales,medium
2,1,4,272,4,Yes,0,Sales,medium
3,3,4,223,5,Yes,0,Sales,low
4,2,3,159,3,Yes,0,Sales,low
5,2,3,153,3,Yes,0,Sales,low
6,1,3,247,4,Yes,0,Sales,low
7,4,4,259,5,Yes,0,Sales,low
8,4,4,224,5,Yes,0,Sales,low
9,2,3,142,3,Yes,0,Sales,low


In [30]:
hr_job_filtered_df = hr_job_filtered_df.rename(columns={'promotion_last_5years': 'YearsSinceLastPromotion'})

def filtering_condition_for_promotion(val):
  if val == 0:
    return 0
  else: 
    return 5
  
hr_job_filtered_df["YearsSinceLastPromotion"] = hr_job_filtered_df["YearsSinceLastPromotion"].apply(lambda x: filtering_condition_for_promotion(x))

In [ ]:
### converting the average_monthly_rate data to determine what is the Monthly_rate, Daily_Rate and Hourly_Rate of the employee

def convert_avg_montly_rate(val):
  
  avg_monthly_hrs = hr_job_filtered_df[val]
  standard_working_days = 22 # assuming 22 days working a month excluding leaves taken
  standard_working_hrs = 8 # 8 hrs/day
  
  
  pass


